In [1]:
!nvidia-smi

Mon Feb 26 08:56:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install transformers
!pip install langchain
!pip install accelerate
!pip install bitsandbytes
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.4/246.4 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.3 MB/s eta 0:00:00


In [3]:
import torch
from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

In [4]:
alpaca_model = LlamaForCausalLM.from_pretrained(
    "chavinlo/alpaca-native",
    device_map='auto',
    load_in_8bit=True,
)

alpaca_tokenizer = LlamaTokenizer.from_pretrained("chavinlo/alpaca-native")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.88G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/335 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [5]:
pipe_param = pipeline(
    "text-generation",
    model=alpaca_model,
    tokenizer=alpaca_tokenizer,
    max_length=450,
    top_p=0.95,
    temperature=0.4,
    repetition_penalty=1.2
)

llm_pipe = HuggingFacePipeline(pipeline=pipe_param)

In [6]:
template = """
Write a SQL Query given the table name {Table} and columns as a list {Columns} for the given question :
{question}.
"""

prompt = PromptTemplate(template=template, input_variables=["Table","question","Columns"])
llm_chain = LLMChain(prompt=prompt, llm=llm_pipe)

def get_llm_output(tble,question,cols):
    llm_chain = LLMChain(prompt=prompt,
                         llm=llm_pipe
                         )
    llm_output = llm_chain.run({"Table" : tble,"question" :question, "Columns" : cols})
    return(llm_output)

In [7]:
tble = "tblOPPERTUNITIESTest"
cols = [
    "OppertunityID",
    "OppertunityIDForSearch",
    "OppertunityName",
    "Created",
    "ESTIMATE",
    "ESTCLOSE_DATE",
    "YearQuarter",
    "Antcipated",
    "SalesRep",
    "SEGMENT_UID",
    "Segment",
    "ACCOUNT_UID",
    "AccountName",
    "SALESSTAGE_UID",
    "Stage",
    "ProjectStage",
    "SQUAREFOOTAGE",
    "ProductLevelId",
    "Probability",
    "SamplesSent",
    "Specified",
    "InvoicePrice",
    "QuanitityPrice",
    "SubmittalsStatus",
    "OutOfRegion",
    "Notes",
    "LastUpdatedBy",
    "LastUpdatedDate",
    "LastUpdatedDateFromPowerApp",
    "Description",
    "PremierFlag",
    "AddressID",
    "LostReasonTypeId",
    "ADDRESS1",
    "ADDRESS2",
    "CITY",
    "STATE",
    "COUNTRYCODE",
    "POSTALCODE",
    "ContactId",
    "NationalAccount",
    "TMAssigned",
    "MaterialBank",
    "NationalAccountName"
]
question = "Find out opportunity name which has invoicePrice between 5000 and 10000"

output_command = get_llm_output(tble,question,cols)
print(output_command)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


SELECT OpportunityName FROM tblOPPERTUNITIESTest WHERE InvoicePrice BETWEEN 5000 AND 10000;
